In [ ]:
import pandas as pd
from long_covid.colors import flatuicolors
from long_covid import styling
from matplotlib import pyplot as plt
from scipy import stats

In [ ]:
# Globally accessible input data
COHORTS = pd.read_feather('../data/03_derived/user_cohorts.feather')
WEEKLY_DEVIATIONS = pd.read_feather('../data/03_derived/weekly_vital_deviations_per_user.feather')

In [ ]:
def trim_axs(axs, N):
    """
    Reduce *axs* to *N* Axes. All further Axes are removed from the figure.
    """
    axs = axs.flat
    for ax in axs[N:]:
        ax.remove()
    return axs[:N]


def QQplot(cohort, color, vitalid, max_week, output_path):
    
    size = 10.5
    
    if max_week == 15:
        f, axarr = plt.subplots(5, 4, figsize=(size, size))
        axarr = trim_axs(axarr, 3 + max_week + 1)
    
    elif max_week == 12:
        f, axarr = plt.subplots(4, 4, figsize=(size, size * 0.83), sharex=True, sharey=True)

    users = COHORTS[COHORTS.unvaccinated].user_id.values
    data = WEEKLY_DEVIATIONS[WEEKLY_DEVIATIONS.userid.isin(users) & (WEEKLY_DEVIATIONS.vitalid == vitalid)]

    for week, ax in zip(range(-3, max_week+1), axarr.flatten()):
        stats.probplot(data[data.weeks_since_test == week].vital_change.values, dist="norm", plot=ax)
        styling.hide_and_move_axis(ax)

        ax.set_ylabel("Sample quantiles")
        ax.set_xlabel("Normal theoretical quantiles")
        
        if week > 1:
            ax.set_title(f"{week} weeks since test", size=10)
        elif week < -1:
            week = abs(week)
            ax.set_title(f"{week} weeks before test", size=10)
        elif week == 1:
            ax.set_title(f"{week} week since test", size=10)
        elif week == -1:
            week = abs(week)
            ax.set_title(f"{week} week before test", size=10)            
        else:
            ax.set_title(f"Week of test", size=10)

        ax.get_lines()[0].set_markerfacecolor(color)
        ax.get_lines()[0].set_markeredgecolor(color)
        ax.get_lines()[0].set_markersize(4)
        ax.get_lines()[1].set_color('k')
       
    plt.tight_layout()
    plt.savefig(output_path)

In [ ]:
QQplot('negative', flatuicolors.belizehole, 65, 15, '../output/qq_negative_rhr.pdf')
QQplot('negative', flatuicolors.belizehole, 9, 15, '../output/qq_negative_steps.pdf')
QQplot('negative', flatuicolors.belizehole, 43, 15, '../output/qq_negative_sleep.pdf')

QQplot('vaccinated', flatuicolors.amethyst, 65, 12, '../output/qq_vaccinated_rhr.pdf')
QQplot('vaccinated', flatuicolors.amethyst, 9, 12, '../output/qq_vaccinated_steps.pdf')
QQplot('vaccinated', flatuicolors.amethyst, 43, 12, '../output/qq_vaccinated_sleep.pdf')

QQplot('unvaccinated', flatuicolors.pomegranate, 65, 15, '../output/qq_unvaccinated_rhr.pdf')
QQplot('unvaccinated', flatuicolors.pomegranate, 9, 15, '../output/qq_unvaccinated_steps.pdf')
QQplot('unvaccinated', flatuicolors.pomegranate, 43, 15, '../output/qq_unvaccinated_sleep.pdf')